In [1]:
# LOAD LIBRARIES
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# USE KERAS WITH DEFAULT TENSORFLOW BACKEND\
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.datasets import mnist

# Load MNIST's 60,000 training images

In [7]:
# LOAD MNIST DATASET AS 60K TRAIN AND 10K TEST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [8]:
# PREPARE DATA FOR NEURAL NETWORK
X_train = x_train / 255.0
X_test = x_test / 255.0
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)
Y_train = to_categorical(y_train, num_classes = 10)

# Generate 25 million more images!!
by randomly rotating, scaling, and shifting MNIST's 60,000 training images.

In [9]:
# CREATE MORE IMAGES WITH DATA AUGMENTATION
datagen = ImageDataGenerator(
        rotation_range=15,
        zoom_range = 0.15,
        width_shift_range=0.1,
        height_shift_range=0.1)

# Build 7 Convolutional Neural Networks

In [10]:
# BUILD CONVOLUTIONAL NEURAL NETWORKS
nets = 7
model = [0] *nets
for j in range(nets):
    model[j] = Sequential()

    model[j].add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(128, kernel_size = 4, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Flatten())
    model[j].add(Dropout(0.4))
    model[j].add(Dense(10, activation='softmax'))

    # COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train 7 CNN

In [11]:
# Learning Rate Scheduler: 매 epoch마다 learning rate를 0.95배로 감소
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

# TRAIN CNNs AND DISPLAY ACCURACIES
epochs = 40
history = [0] * nets
results = [0] * nets

for j in range(nets):
    # 데이터를 훈련 세트와 검증 세트로 분할
    X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size=0.1)

    # CNN 훈련 및 진행 상태 출력
    history[j] = model[j].fit(datagen.flow(X_train2, Y_train2, batch_size=64),
                              epochs=epochs,
                              steps_per_epoch=X_train2.shape[0] // 64,
                              validation_data=(X_val2, Y_val2),
                              callbacks=[annealer],
                              verbose=1)  # 진행 상황을 출력하도록 verbose=1 설정

    # 훈련 및 검증 정확도 출력
    print("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        j + 1,
        epochs,
        history[j].history['accuracy'][epochs - 1],  # 'acc' 대신 'accuracy'로 수정
        history[j].history['val_accuracy'][epochs - 1]  # 'val_acc' 대신 'val_accuracy'로 수정
    ))

    # PREDICT DIGITS FOR CNN J ON MNIST 10K TEST
    results[j] = model[j].predict(X_test)
    results2 = np.argmax(results[j], axis=1)

    # CNN J의 MNIST 10K 테스트 정확도 계산
    c = 0
    for i in range(10000):
        if results2[i] != y_test[i]:
            c += 1
    print("CNN %d: Test accuracy = %f" % (j + 1, 1 - c / 10000.))

Epoch 1/40
843/843 ━━━━━━━━━━━━━━━━━━━━ 42s 38ms/step - accuracy: 0.7462 - loss: 0.8436 - val_accuracy: 0.9855 - val_loss: 0.0514 - learning_rate: 0.0010
Epoch 2/40
843/843 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - accuracy: 0.9062 - loss: 0.1893 - val_accuracy: 0.9857 - val_loss: 0.0515 - learning_rate: 9.5000e-04
Epoch 3/40
843/843 ━━━━━━━━━━━━━━━━━━━━ 25s 29ms/step - accuracy: 0.9605 - loss: 0.1316 - val_accuracy: 0.9878 - val_loss: 0.0380 - learning_rate: 9.0250e-04
Epoch 4/40
843/843 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - accuracy: 0.9531 - loss: 0.1362 - val_accuracy: 0.9887 - val_loss: 0.0380 - learning_rate: 8.5737e-04
Epoch 5/40
843/843 ━━━━━━━━━━━━━━━━━━━━ 39s 28ms/step - accuracy: 0.9691 - loss: 0.1003 - val_accuracy: 0.9880 - val_loss: 0.0403 - learning_rate: 8.1451e-04
Epoch 6/40
843/843 ━━━━━━━━━━━━━━━━━━━━ 1s 784us/step - accuracy: 1.0000 - loss: 0.0348 - val_accuracy: 0.9883 - val_loss: 0.0404 - learning_rate: 7.7378e-04
Epoch 7/40
843/843 ━━━━━━━━━━━━━━━━━━━━ 24s 28ms/step - 

# Ensemble 7 CNN and Predict

In [12]:
# PREDICT DIGITS FOR ENSEMBLE ON MNIST 10K TEST
results2 = np.zeros( (X_test.shape[0],10) )
for j in range(nets):
    results2 = results2 + results[j]
results2 = np.argmax(results2,axis = 1)

# CALCULATE ACCURACY OF ENSEMBLE ON MNIST 10K TEST SET
c=0
for i in range(10000):
    if results2[i]!=y_test[i]:
        c +=1
print("Ensemble Accuracy = %f" % (1-c/10000.))

Ensemble Accuracy = 0.997200
